In [ ]:
from diffusers import StableDiffusionXLPipeline, KDPM2AncestralDiscreteScheduler, AutoencoderKL
import torch

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16
)
pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")


In [ ]:
import spacy
import nltk
import json
import re
import torch
import numpy as np
from collections import OrderedDict
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import MarianMTModel, MarianTokenizer

##############################################################################
# Load translation model
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Arabic detection regex
arabic_char_pattern = re.compile(r'[\u0600-\u06FF]')

def is_arabic(text):
    return len(arabic_char_pattern.findall(text)) > 3

def translate_arabic_to_english_preserve_english(text):
    if not is_arabic(text):
        return text

    english_words = re.findall(r'[a-zA-Z0-9@#_.+-]+', text)
    placeholder_map = {}
    for i, word in enumerate(english_words):
        placeholder = f"ENG{i}"
        text = text.replace(word, placeholder)
        placeholder_map[placeholder] = word

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    for placeholder, word in placeholder_map.items():
        translated_text = translated_text.replace(placeholder, word)

    return translated_text
##############################################################################
# Load NLP & Sentiment Models
nlp = spacy.load("en_core_web_sm")
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
sentiment_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Load color dataset
color_dataset_path = "/kaggle/input/ccoolloorrss/colornames_modified.json"
try:
    with open(color_dataset_path, "r") as f:
        color_data = json.load(f)
        color_names = {entry["name"].lower() for entry in color_data}
except FileNotFoundError:
    print(f"Error: {color_dataset_path} not found. Using default color set.")
    color_names = set()

# --- Shared Sentiment Analysis ---
def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    sentiment_scores = torch.softmax(outputs.logits, dim=1).numpy().flatten()
    sentiment_labels = ["negative", "neutral", "positive"]
    sentiment_index = np.argmax(sentiment_scores)
    base_mood = sentiment_labels[sentiment_index]

    mood_mapping = {
        "negative": [
            "subtle and understated", "muted and elegant", "dark and dramatic",
            "reserved and serious", "moody and introspective", "mysterious and sophisticated"
        ],
        "neutral": [
            "balanced and harmonious", "sophisticated and refined", "modern and professional",
            "minimalist and clean", "sleek and polished", "contemporary and timeless"
        ],
        "positive": [
            "bold and energetic", "vibrant and dynamic", "playful and cheerful",
            "expressive and artistic", "friendly and inviting", "joyful and captivating"
        ],
    }

    confidence_threshold = 0.6
    if sentiment_scores[sentiment_index] > confidence_threshold:
        selected_mood = np.random.choice(mood_mapping[base_mood])
    else:
        selected_mood = "eclectic and expressive"

    if sentiment_scores[0] > 0.3 and sentiment_scores[2] > 0.3:
        selected_mood = "contrasting and experimental"
    elif sentiment_scores[1] > 0.5:
        selected_mood = "timeless and balanced"

    return selected_mood

def get_output_style(design_type, mood):
    vector_designs = {
        "logo", "app icon", "business card", "branding", "character design", "product label", "slogan" 
    }
    raster_designs = {
        "poster", "album cover", "banner", "sticker", "mockup", "advertisement", "flyer", "billboard", "cinema poster", "movie poster", "campaign poster"
    }
    print_designs = {
        "post card", "invitation", "brochure", "menu design", "book cover"
    }
    digital_ui = {
        "social media post", "UI/UX design", "e-commerce product image", "NFT art", "infographic"
    }

    # Base logic
    if design_type in vector_designs:
        return "vector-based, scalable design"
    elif design_type in raster_designs:
        return "high-resolution raster image"
    elif design_type in print_designs:
        return "print-ready layout (CMYK, 300 DPI)"
    elif design_type in digital_ui:
        return "digital-optimized, responsive layout"
    else:
        # Optional fallback based on mood
        if "minimalist" in mood or "clean" in mood:
            return "flat design, minimal elements"
        elif "artistic" in mood or "dynamic" in mood:
            return "expressive, layered illustration"
        else:
            return "versatile mixed-media style"

# --- Extract Keywords Version 1 (for logos) ---
def extract_logo_keywords_v1(user_input):
    doc = nlp(user_input)
    capitalized_words = [token.text for token in doc if token.text.istitle() or token.text.isupper()]
    adjectives = [token.text.lower() for token in doc if token.pos_ == "ADJ"]
    nouns = [token.text.lower() for token in doc if token.pos_ == "NOUN"]
    theme = " ".join(adjectives[:10]) if adjectives else "unique and creative"
    combined = capitalized_words + nouns
    elements = ", ".join(combined[:10]) if combined else "unique and creative"

    words = re.findall(r"\b[a-zA-Z]+\b", user_input.lower())
    unique_color_words = list({word for word in words if word in color_names})
    hex_colors = re.findall(r"#(?:[0-9a-fA-F]{3}){1,2}\b", user_input)
    all_colors = unique_color_words + hex_colors

    mood = analyze_sentiment(user_input)

    requested = next((
        label for label in [
            "logo", "slogan", "poster", "post card", "advertisement", "business card", "book cover", "sticker", "banner",
            "flyer", "brochure", "album cover", "packaging", "social media post", "NFT art", "app icon",
            "merchandise", "invitation", "billboard", "infographic", "mockup", "e-commerce product image",
            "UI/UX design", "menu design", "advertising campaign", "character design", "e-book cover",
            "business branding", "product label", "campaign poster"
        ] if label in user_input.lower()
    ), "general design")

    return {
        "requested": requested,
        "theme": theme,
        "mood": mood,
        "elements": elements,
        "color_scheme": ", ".join(all_colors) if all_colors else "default palette",
        "output_style": get_output_style(requested_type, mood),
    }

# --- Extract Keywords Version 2 (for other types) ---
def extract_keywords(user_input):
    """Extracts relevant keywords dynamically using NLP and dependency parsing."""
    doc = nlp(user_input)

    # Define the design types
    design_types = {
        "logo", "poster", "post card", "advertisement", "business card", "movie poster",
        "book cover", "sticker", "banner", "flyer", "brochure", "album cover", "card",
        "packaging", "social media post", "NFT art", "app icon", "merchandise", "character design",
        "invitation", "billboard", "infographic", "mockup", "cinema poster", "game poster",
        "menu design", "campaign poster", "branding", "product label", "view"
    }

    # Detect requested type
    requested_type = next((design for design in design_types if design in user_input.lower()), "realistic world design")

    # Common: Extract colors
    words = re.findall(r"\b[a-zA-Z]+\b", user_input.lower())
    unique_color_words = list({word for word in words if word in color_names})
    hex_colors = re.findall(r"#(?:[0-9a-fA-F]{3}){1,2}\b", user_input)
    all_colors = unique_color_words + hex_colors

    # Common: Sentiment analysis
    mood = analyze_sentiment(user_input)

    if "logo" in requested_type.lower():
        # Extract adjectives for themes
        adjectives = [token.text.lower() for token in doc if token.pos_ == "ADJ"]

        # Extract nouns, proper nouns, and verbs for elements
        elements = [token.text.lower() for token in doc if token.pos_ in ["NOUN", "PROPN", "VERB"]]

        # Extract capitalized words (words that start with a capital letter or are fully capitalized)
        capitalized_words = [token.text for token in doc if token.text.istitle() or token.text.isupper()]

        # Combine theme and element keywords, prioritizing capitalized words
        combined_theme = adjectives + capitalized_words  # Prioritizing capitalized words
        theme = " ".join(combined_theme[:10]) if combined_theme else "unique and creative"

        # Combine all element keywords, prioritizing capitalized words
        combined_elements = capitalized_words + elements
        elements = " ".join(combined_elements[:10]) if combined_elements else "unique and creative"


    else:
        # Second version logic (Non-logo)
        adjectives = [token.text.lower() for token in doc if token.pos_ == "ADJ" and token.text.lower() not in color_names]
        theme = " ".join(adjectives) if adjectives else "unique and creative"

        elements = []
        for chunk in doc.noun_chunks:
            if chunk.root.text.lower() not in stop_words:
                elements.append(chunk.text.lower())
        elements = list(OrderedDict.fromkeys(elements))
        elements = ", ".join(elements) if elements else "abstract shapes"

    return {
        "requested": requested_type,
        "theme": theme,
        "mood": mood,
        "elements": elements,
        "color_scheme": ", ".join(all_colors) if all_colors else "default palette",
        "output_style": get_output_style(requested_type, mood),
    }


def generate_logo_prompt(brand_name, brand_style, user_input):
    """Generates a logo or design prompt based on extracted keywords."""
    keywords = extract_keywords(user_input)

    return (
        f'{keywords["requested"]} for "{brand_name}" with ({brand_style} style). '
        f'It should reflect a {keywords["theme"]} theme and evoke a {keywords["mood"]} feeling. '
        f'Incorporate {keywords["elements"]} while using a {keywords["color_scheme"]} color palette. '
        f'The final design must be {keywords["output_style"]}.'
    )


# --- Optional Redundancy Cleaner ---
def remove_redundancy(prompt):  
    words = prompt.split()
    unique_words = list(OrderedDict.fromkeys(words))
    prompt_cleaned = " ".join(unique_words)
    prompt_cleaned = re.sub(r"\b(\w+ and \w+)\b(?=.*\b\1\b)", r"\1", prompt_cleaned)
    return prompt_cleaned


In [ ]:
!pip install pyngrok
!pip install flask-cors
!ngrok config add-authtoken 2u3LkoKYpHkC0CawsxkYP3KZtp7_6CoGa2Gf6WXPraZg9ABoT

In [ ]:
from flask import Flask, request, send_file
from flask_cors import CORS
from pyngrok import ngrok
from PIL import Image
import torch

# Assumes you have these functions and model pipeline defined somewhere
# from your_module import generate_logo_prompt, remove_redundancy, pipe

app = Flask(__name__)
CORS(app)

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prmp = data.get("prompt", "")
    brandName = data.get("brandName", "")
    brandStyle = data.get("brandStyle", "")

    raw_prompt = generate_logo_prompt(brandName, brandStyle, translate_arabic_to_english_preserve_english(prmp))
    prompt = remove_redundancy(raw_prompt)

    print("Brand Name:", brandName)
    print("-----------------------------------------------------------------------------")
    print("Brand Style:", brandStyle)
    print("-----------------------------------------------------------------------------")
    print("Original:", prmp)
    print("-----------------------------------------------------------------------------")
    print("Prompt:", prompt)

    image = pipe(
        prompt=prompt,
        negative_prompt=(
            "low quality, blurry, distorted, blurry text, unreadable letters, distorted words, "
            "low resolution, messy typography, misspelled text, cut-off letters, overlapping text, "
            "incorrect spacing, warped text, illegible handwriting, pixelated text, fuzzy edges, "
            "text artifacts, stretched letters, chaotic layout, unclear details, inconsistent font style, "
            "deformed words, broken letters, random symbols, gibberish text"
        ),
        width=1024,
        height=1024,
        guidance_scale=15,
        num_inference_steps=100
    ).images[0]

    image.save("generated.png")
    return send_file("generated.png", mimetype='image/png')


if __name__ == "__main__":
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel available at:", public_url)
    print(" * Running on http://127.0.0.1:5000")

    # Run the Flask app
    app.run()
